In [1]:
# create data generator
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras import backend as K
#  to split the data of training and testing sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
batch_size = 128
num_classes = 10
epochs = 10
input_shape=(28,28,1)
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [6]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# conversion of class vectors to matrices of  binary class 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")



In [ ]:
model.save('digit_model.h5')
model.save('digit_model.keras')
print("Saving the model as digit_model.keras.h5 and digit_model.keras")

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [32]:
import gradio as gr 
from sklearn.neural_network import MLPClassifier 
import torchvision.datasets as datasets 
import seaborn as sns 
import pickle

#dark mode seaborn 
sns.set_style("darkgrid")

In [19]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)

In [21]:
print(mnist_trainset.data.shape)
print(mnist_testset.data.shape)
print(mnist_trainset.targets.shape)
print(mnist_testset.targets.shape)

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])
torch.Size([60000])
torch.Size([10000])


In [22]:
X_train = mnist_trainset.data
y_train = mnist_trainset.targets
X_test = mnist_testset.data
y_test = mnist_testset.targets

In [23]:
X_train = X_train.numpy() 
X_test = X_test.numpy()
y_train = y_train.numpy()
y_test = y_test.numpy()

In [24]:
#plot the first image 
sns.heatmap(X_train[0], cmap = "gray")

<Axes: >

In [25]:
X_train = X_train.reshape(60000, 784)/255.0
X_test = X_test.reshape(10000, 784)/255.0

In [26]:
#train the model 
mlp = MLPClassifier(hidden_layer_sizes=(32,32))
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(32, 32))

In [27]:
#print the accuracies 
print("Training Accuracy: ", mlp.score(X_train, y_train))
print("Testing Accuracy: ", mlp.score(X_test, y_test))

Training Accuracy:  1.0
Testing Accuracy:  0.9666


In [39]:
pickle.dump(mlp, open("digitmodel.sav", 'wb'))


In [40]:
loaded_model = pickle.load(open("digitmodel.sav", 'rb'))

In [41]:
print("Testing Accuracy: ", loaded_model.score(X_test, y_test))

Testing Accuracy:  0.9666


In [42]:
def predict(img):
    img = img.reshape(1,784)/255.0
    prediction = loaded_model.predict(img)[0]
    return int(prediction)

In [43]:
gr.Interface(fn= predict, inputs = "sketchpad", outputs ="label", interactive=True).launch()


C:\Users\ALOK\AppData\Local\Temp\ipykernel_17628\4054000665.py:1: GradioUnusedKwargWarning: You have unused kwarg parameters in Interface, please remove them: {'interactive': True}
  gr.Interface(fn= predict, inputs = "sketchpad", outputs ="label", interactive=True).launch()


Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
